<a href="https://colab.research.google.com/github/shaswatgithub/stamatics2025-deepflow-/blob/main/sale_price_of_bulldozers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


train_df = pd.read_csv('/content/Train.csv')
test_df = pd.read_csv('/content/Test.csv')

print("Train Shape:", train_df.shape)
print("Test Shape:", test_df.shape)
train_df.head()


Train Shape: (214640, 53)
Test Shape: (12457, 52)


,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:

train_df['saledate'] = pd.to_datetime(train_df['saledate'])
test_df['saledate'] = pd.to_datetime(test_df['saledate'])

train_df.sort_values('saledate', inplace=True)


missing = train_df.isnull().sum() / len(train_df)
missing[missing > 0.3].sort_values(ascending=False)


train_df.drop(columns=missing[missing > 0.3].index, inplace=True)
test_df.drop(columns=missing[missing > 0.3].index, inplace=True)

drop_cols = ['SalesID', 'MachineID', 'ModelID', 'saledate']
train_df = train_df.drop(columns=drop_cols)
test_ids = test_df['SalesID']
test_df = test_df.drop(columns=drop_cols)


for col in train_df.select_dtypes(include=np.number).columns:
    train_df[col].fillna(train_df[col].median(), inplace=True)

for col in test_df.select_dtypes(include=np.number).columns:
    test_df[col].fillna(train_df[col].median(), inplace=True)


from sklearn.preprocessing import LabelEncoder

for col in train_df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    full_col = pd.concat([train_df[col], test_df[col]], axis=0).astype(str)
    le.fit(full_col)
    train_df[col] = le.transform(train_df[col].astype(str))
    test_df[col] = le.transform(test_df[col].astype(str))


train_df.head()


<ipython-input-2-3013847157>:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df[col].fillna(train_df[col].median(), inplace=True)
<ipython-input-2-3013847157>:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

,SalePrice,datasource,auctioneerID,YearMade,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Enclosure,Hydraulics
205615,9500,132,18,1974,2911,1166,123,45,44,4,4,5,0
205752,10500,132,99,1973,556,189,123,10,9,3,3,0,11
55455,17000,132,99,1972,64,14,34,34,9,1,1,5,4
119369,120000,132,99,1986,1741,547,38,48,9,4,4,2,0
142491,21000,132,99,1972,1737,547,34,48,9,4,4,5,0


In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split

X = train_df.drop('SalePrice', axis=1)
y = train_df['SalePrice']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)


model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)


val_preds = model.predict(X_valid)
rmsle = np.sqrt(mean_squared_log_error(y_valid, val_preds))
print("Validation RMSLE:", rmsle)


Validation RMSLE: 0.2932935702133417


In [4]:

final_preds = model.predict(test_df)


output = pd.DataFrame({
    'SalesID': test_ids,
    'SalePrice': final_preds
})

output.to_csv('test_predictions.csv', index=False)
output.head()


,SalesID,SalePrice
0,1227829,13592.277778
1,1227844,27962.000000
2,1227847,52978.142857
3,1227848,86228.333333
4,1227863,31068.952381
